In [5]:
from dataclasses import dataclass
from math import gcd, ceil
import re
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest, batched, pairwise
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, sortdict, data, quantify, atom, atoms, list_atoms, list_multiply, mapt, Node
from aocutils.grid import TURNRIGHT, TURNLEFT, move, gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, ranges2grid, manhattan, conv1d, conv2d, rotate, rot90
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst, shoelace
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot, animate
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard
E = (0,1); W = (0,-1); N = (-1,0); S = (1,0); NE = (-1,1); SE = (1,1); SW = (1,-1); NW = (-1,-1)
winds = [N, E, S, W]
allwinds = [N, NE, E, SE, S, SW, W, NW]
inf = float('inf')

In [223]:
class Range():
    # helper class to compare ranges (start, end)
    def __init__(self, start, end):
        self.start = start
        self.end = end
    def __iter__(self):
        return iter((self.start, self.end))
    def __getitem__(self, key):
        if key == 0:
            return self.start
        else:
            return self.end
    def __lt__(self, other):
        return self.end < other[0]
    def __gt__(self, other):
        return self.start > other[1]
    def __le__(self, other):
        return self.end <= other[0]  
    def __ge__(self, other):    
        return self.start >= other[1]
    def __eq__(self, other):
        return self.start == other.start and self.end == other.end
    def contains(self, other):
        return self.start <= other[0] and self.end >= other[1]
    def __repr__(self):
        return f'({self.start}, {self.end})'
    def has_left_overlap(self, other):
        return self.start < other[1] and self.end > other[1]
    def has_right_overlap(self, other):
        return self.start < other[0] and self.end > other[0]
    def overlaps(self, other):
        return self.start <= other.end and other.start <= self.end
    def fully_overlaps(self, other):
        return self.start <= other.start and self.end >= other.end
    def get_overlap(self, other):
        if self.overlaps(other):
            return (max(self.start, other[0]), min(self.end, other[1]))
        else:
            return None
    def get_gap(self, other):
        if self < other:
            return (self.end, other[0])
        elif self > other:
            return (other[1], self.start)
        else:
            return None
assert Range(2,4) < (5,6)
assert Range(5,6) > (2,4)


    

In [245]:
class Range():
    # helper class to compare ranges (start, end)
    def __init__(self, start, end):
        self.start = start
        self.end = end
    def __iter__(self):
        return iter((self.start, self.end))
    def __getitem__(self, key):
        if key == 0:
            return self.start
        else:
            return self.end
    def __lt__(self, other):
        return self.end < other[0]
    def __gt__(self, other):
        return self.start > other[1]
    def __le__(self, other):
        return self.end <= other[0]  
    def __ge__(self, other):    
        return self.start >= other[1]
    def __eq__(self, other):
        return self.start == other[0] and self.end == other[1]
    def contains(self, other):
        return self.start <= other[0] and self.end >= other[1]
    def __repr__(self):
        return f'({self.start}, {self.end})'
    def has_left_overlap(self, other):
        return other[0] <= self.start <= other[1]
    def has_right_overlap(self, other):
        return other[0] <= self.end <= other[1]
    def overlapscorrect(self, other):
        return self.start <= other.end and other.start <= self.end
    def overlaps(self, other):
        return self.has_left_overlap(other) or self.has_right_overlap(other) or self == other
    
    def get_overlap(self, other):
        if self.overlaps(other):
            return (max(self.start, other[0]), min(self.end, other[1]))
        else:
            return None
    def get_gap(self, other):
        if self < other:
            return (self.end, other[0])
        elif self > other:
            return (other[1], self.start)
        else:
            return None
assert Range(2,4) < (5,6)
assert Range(5,6) > (2,4)


# | export
def range_full_overlap(a,b):
    # fully contains. Also returns True if a==b
    # ranges are inclusive
    start, end = a
    start2, end2 = b
    ans = max(start,start2) <= min(end, end2)
    if ans[1] > ans[0]:
        return None
    return ans

def range_some_overlap(a,b):
    # partly overlap
    # ranges are inclusive
    start, end = a
    start2, end2 = b
    return (start <= start2 and end >= end2) or start2 <= start and end2 >= end

def range_full_overlap2(a,b):
    # fully contains. Also returns True if a==b
    # ranges are inclusive
    start, end = a
    start2, end2 = b
    return not (end < start2 or start > end2)

a = (2,5); b = (1,105)
range_full_overlap2(a,b),range_some_overlap(a,b), range_full_overlap2(a,b)
    

    

(True, True, True)

In [249]:
ranges, ing = open("in.txt").read().strip().split("\n\n")
lines = [line for line in ranges.split('\n')]

r = []
for line in lines:
    a,b = line.split('-')
    a, b = int(a), int(b)
    r.append(Range(a,b))
ans = 0
for i in ing.split('\n'):
    num = int(i)
    for ran in r:
        if ran.overlaps(Range(num,num)):
            ans += 1
            break
print(ans)
print
ranges = r



0


In [250]:

removed = True
while removed:
    print('do', len(ranges))
    removed=False
    newranges = []
    for r in ranges:
        for nr in newranges[:]:
            if r.overlaps(nr):
                if not r.overlaps(nr):
                    print('error', r, nr, nr.overlaps(r))
                newr = Range(min(nr.start, r.start), max(nr.end, r.end))
                print('overlap', r, nr, newr)
                newranges.remove(nr)
                newranges.append(newr)
                removed = True
                break
        else:
            newranges.append(r)
    ranges = newranges
ans = 0
points = defaultdict(int)
for cr in ranges:
    ans += cr.end - cr.start + 1
    if cr.start != cr.end:
        points[cr.start] += 1
        points[cr.end] += 1
    else:
        points[cr.start] += 1
    if 410238139187238  in [cr.end, cr.start]:
        print('found', cr)
    
ans

do 177
overlap (409482784320772, 410238139187238) (410238139187238, 410761947885171) (409482784320772, 410761947885171)
overlap (473585443607145, 476193509708588) (473585443607145, 480753922496691) (473585443607145, 480753922496691)
overlap (108955695016421, 110491259771069) (108216544203392, 109266157783449) (108216544203392, 110491259771069)
overlap (103400283150965, 104688372776733) (102512629946737, 104062192307677) (102512629946737, 104688372776733)
overlap (78488449406874, 79127270522699) (78488449406874, 79312644451028) (78488449406874, 79312644451028)
overlap (162137806986087, 162730635445893) (162730635445893, 163507945304068) (162137806986087, 163507945304068)
overlap (393503713708521, 394980020153879) (392221539403379, 393743951722566) (392221539403379, 394980020153879)
overlap (135718199937845, 139039399766737) (132568360863761, 139039399766737) (132568360863761, 139039399766737)
overlap (193159859342438, 196922816259798) (193985197063188, 199203019947316) (193159859342438,

341553633059986